In [1]:
import os
gpu_device = 0    # número identificador del device puede ser: 0, 1, 2, o 3
os.environ["CUDA_VISIBLE_DEVICES"]=str(gpu_device)
from numba import cuda
cuda.select_device(0)

<weakproxy at 0x7f166c1ecf40; to 'numba.cuda.cudadrv.driver.Device' at 0x7f166c1a1d30>

In [2]:
import pandas as pd
import re
import random
import numpy as np
import math
import pickle
from sklearn.metrics import classification_report, accuracy_score
from datasets import Dataset,concatenate_datasets,load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%run ../utils/data.ipynb
%run ../utils/Generatives-small_data-answer2.ipynb

In [4]:

# Cargar modelo y tokenizador
model_name = "microsoft/biogpt"
tokenizer = AutoTokenizer.from_pretrained(model_name)

inputs,targets,val_data,examples_raw = flatten_and_filter_dataset(ground_truth,report_data)  

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    # evaluation_strategy="epoch",
    learning_rate=5e-5,  # Lower for fine-tuning without losing generalization
    fp16=True,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    
    num_train_epochs=7,  # Shorter fine-tuning stage
    weight_decay=0.01,  # Lower weight decay to preserve learned features
    save_total_limit=2,
)

training_args_all = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    # evaluation_strategy="epoch",
    learning_rate=5e-6,  # Lower for fine-tuning without losing generalization
    fp16=True,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    
    num_train_epochs=5,  # Shorter fine-tuning stage
    weight_decay=0.01,  # Lower weight decay to preserve learned features
    save_total_limit=2,
)

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
dataset_final=pd.DataFrame.from_dict(inputs,orient='index')
targets=pd.DataFrame.from_dict(targets,orient='index')
val_data=pd.DataFrame.from_dict(val_data,orient='index')
dataset_final.columns=["text"]
targets.columns=["label"]
val_data.columns=["val_data"]
dataset_final=pd.concat([dataset_final, val_data],axis=1)

In [6]:
accuracies,output_dic=cross_validation(dataset_final,targets,training=True,low_beams=False,testing=False)

fold 0
                                                                                                 text  \
001-323-702-20221219-120748.390_age                 Question: does the patient's age appear in the...   
001-323-702-20221219-120748.390_birads              Question: what is the final BI-RADS classifica...   
001-323-702-20221219-120748.390_calcifications_...  Question: does the following breast medical re...   
001-323-702-20221219-120748.390_density_echo        Question: what is the breast density found in ...   
001-323-702-20221219-120748.390_density_mammo       Question: what is the breast density found in ...   
...                                                                                               ...   
046-630-291-20230619-112356_age                     Question: does the patient's age appear in the...   
046-630-291-20230619-112356_family                  Question: does the patient have any family his...   
046-630-291-20230619-112356_history             

Map: 100%|██████████| 405/405 [00:01<00:00, 222.09 examples/s]


Epoch,Training Loss,Validation Loss
1,No log,0.218169
2,No log,0.129653
3,0.315300,0.113166
4,0.315300,0.090985
5,0.096200,0.088176
6,0.041200,0.080307


/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/transformers/generation/configuration_utils.py:677: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


age outputs
['no.', 'no.', 'no.', 'no.', 'no.', 'no.', '29.', 'no.', '48.', '70.', 'no.', '18.', 'no.', 'no.', 'no.', 'no.', '44.', 'no.', 'no.', 'no.']
age labels
['no.', 'no.', 'no.', 'no.', 'no.', 'no.', '29.', 'no.', '48.', '70.', 'no.', '18.', 'no.', 'no.', 'no.', 'no.', '44.', 'no.', 'no.', 'no.']
tipo outputs
['normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', '', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision rep

...

age outputs
['no.', 'no.', '32.', 'no.', '50.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', '42.', 'no.', 'no.', 'no.', 'no.']
age labels
['no.', 'no.', '32.', 'no.', '50.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', '42.', 'no.', 'no.', 'no.', 'no.']
tipo outputs
['normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'biopsy report. Comparison of biopsy', 'normal control or r

Map: 100%|██████████| 396/396 [00:01<00:00, 207.39 examples/s]


Epoch,Training Loss,Validation Loss
1,No log,0.187947
2,No log,0.136871
3,0.320600,0.129667
4,0.320600,0.109831
5,0.093100,0.099835
6,0.093100,0.100018
7,0.036200,0.101700


/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/transformers/generation/configuration_utils.py:677: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


age outputs
['50.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', '86.', 'no.', 'no.', 'no.', 'no.', 'no.', '59.', 'no.']
age labels
['50.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', 'no.', '86.', 'no.', 'no.', 'no.', 'no.', 'no.', '59.', 'no.']
tipo outputs
['biopsy report.) biopsy report', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'normal control or revision report.', 'nodal staging ultrasound report. Image'

In [7]:
accuracies

[0.05188679245283019,
 0.0,
 0.0,
 0.010752688172043012,
 0.0,
 0.0,
 0.016129032258064516,
 0.005376344086021506,
 0.0,
 0.0,
 0.0,
 0.005376344086021506,
 0.0,
 0.005376344086021506,
 0.0,
 0.0,
 0.0,
 0.05913978494623656,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [ ]:
first fold
4 epoch 1e-5
Accuracy for age: 1.0000
Accuracy for tipo: 1.0000
Accuracy for tecnica: 1.0000
Accuracy for family: 1.0000
Accuracy for history: 0.9500
Accuracy for symtomatic: 0.9500
Accuracy for prosthesis: 1.0000
Accuracy for birads: 1.0000
Accuracy for density_mammo: 0.9474
Accuracy for density_echo: 0.8947
Accuracy for lymph_suspicious: 1.0000
Accuracy for nodules_echo_num: 0.9474
Accuracy for nodules_echo_margin: 0.8750
Accuracy for nodules_echo_echogenicity: 1.0000
Accuracy for nodules_echo_location: 0.8750
Accuracy for nodules_echo_size: 0.8750
Accuracy for nodules_echo_known: 0.8750
Accuracy for nodules_echo_stable: 1.0000


2 epoch 5e-6
Accuracy for age: 1.0000
Accuracy for tipo: 1.0000
Accuracy for tecnica: 1.0000
Accuracy for family: 1.0000
Accuracy for history: 1.0000
Accuracy for symtomatic: 0.9500
Accuracy for prosthesis: 1.0000
Accuracy for birads: 1.0000
Accuracy for density_mammo: 0.9474
Accuracy for density_echo: 0.8947
Accuracy for lymph_suspicious: 1.0000
Accuracy for nodules_echo_num: 0.9474
Accuracy for nodules_echo_shape: 0.8750
Accuracy for nodules_echo_margin: 0.7500
Accuracy for nodules_echo_echogenicity: 1.0000
Accuracy for nodules_echo_location: 0.5000
Accuracy for nodules_echo_size: 0.8750
Accuracy for nodules_echo_known: 1.0000
Accuracy for nodules_echo_stable: 1.0000

In [ ]:
second fold
Accuracy for age: 1.0000
Accuracy for tipo: 0.9048
Accuracy for tecnica: 1.0000
Accuracy for family: 1.0000
Accuracy for history: 1.0000
Accuracy for symtomatic: 0.8824
Accuracy for prosthesis: 1.0000
Accuracy for birads: 0.8889
Accuracy for density_mammo: 1.0000
Accuracy for density_echo: 1.0000
Accuracy for lymph_suspicious: 1.0000
Accuracy for nodules_echo_num: 0.7778
Accuracy for nodules_echo_margin: 0.8000
Accuracy for nodules_echo_echogenicity: 0.8000
Accuracy for nodules_echo_location: 0.7000
Accuracy for nodules_echo_size: 0.8889
Accuracy for nodules_echo_known: 0.7778
Accuracy for nodules_echo_stable: 0.6000

Accuracy for age: 1.0000
Accuracy for tipo: 0.9048
Accuracy for tecnica: 1.0000
Accuracy for family: 1.0000
Accuracy for history: 1.0000
Accuracy for symtomatic: 0.9412
Accuracy for prosthesis: 1.0000
Accuracy for birads: 0.9444
Accuracy for density_mammo: 1.0000
Accuracy for density_echo: 1.0000
Accuracy for lymph_suspicious: 0.9444
Accuracy for nodules_echo_num: 0.6111
Accuracy for nodules_echo_shape: 0.8000
Accuracy for nodules_echo_margin: 0.6000
Accuracy for nodules_echo_echogenicity: 0.8000
Accuracy for nodules_echo_location: 0.5000
Accuracy for nodules_echo_size: 1.0000
Accuracy for nodules_echo_known: 0.7778
Accuracy for nodules_echo_stable: 0.8000

Accuracy for age: 1.0000
Accuracy for tipo: 0.9524
Accuracy for tecnica: 1.0000
Accuracy for family: 1.0000
Accuracy for history: 0.5882
Accuracy for symtomatic: 0.8235
Accuracy for prosthesis: 1.0000
Accuracy for birads: 0.9444
Accuracy for density_mammo: 1.0000
Accuracy for density_echo: 0.8889
Accuracy for lymph_suspicious: 1.0000
Accuracy for nodules_echo_num: 0.7222
Accuracy for nodules_echo_shape: 0.9000
Accuracy for nodules_echo_margin: 0.7000
Accuracy for nodules_echo_echogenicity: 0.6000
Accuracy for nodules_echo_location: 0.6000
Accuracy for nodules_echo_size: 1.0000
Accuracy for nodules_echo_known: 0.7778
Accuracy for nodules_echo_stable: 0.6000

In [ ]:
third fold
Accuracy for age: 0.9500
Accuracy for tipo: 0.9500
Accuracy for tecnica: 0.9000
Accuracy for family: 1.0000
Accuracy for history: 0.7222
Accuracy for symtomatic: 0.9444
Accuracy for prosthesis: 1.0000
Accuracy for birads: 1.0000
Accuracy for density_mammo: 1.0000
Accuracy for density_echo: 0.8333
Accuracy for lymph_suspicious: 0.9444
Accuracy for nodules_echo_num: 0.8333
Accuracy for nodules_echo_margin: 0.8889
Accuracy for nodules_echo_echogenicity: 0.8889
Accuracy for nodules_echo_location: 0.7778
Accuracy for nodules_echo_size: 1.0000
Accuracy for nodules_echo_known: 0.9000
Accuracy for nodules_echo_stable: 0.8571

In [7]:
for i,tipo in enumerate(questions):
    print(tipo, accuracies[i])

age 0.0
tipo 0.0
tecnica 0.0
family 0.0
history 0.0
symtomatic 0.0
prosthesis 0.0
birads 0.0
density_mammo 0.0
calcifications_benign 0.0
density_echo 0.0
ganglio_mamo 0.0
lymph_benign 0.0
lymph_suspicious 0.0
parenchymal_distortion 0.0
simple_cyst 0.0
ductal_ectasia 0.0
nodules_echo_num 0.0
nodules_echo_description 0.0
nodules_echo_shape 0.0
nodules_echo_margin 0.0
nodules_echo_echogenicity 0.0
nodules_echo_location 0.0
nodules_echo_size 0.0
nodules_echo_known 0.0
nodules_echo_stable 0.0


In [ ]:
age 0.9952830188679245
tipo 0.9716981132075472
tecnica 0.9858490566037735
family 0.989247311827957
history 0.9301075268817204
symtomatic 0.0
prosthesis 0.9946236559139785
birads 0.967741935483871
density_mammo 0.9516129032258065
density_echo 0.6935483870967742
lymph_suspicious 0.9731182795698925
nodules_echo_num 0.7849462365591398
nodules_echo_size 0.7349397590361446
nodules_echo_known 0.7831325301204819
nodules_echo_stable nan

In [13]:
questions

['age',
 'tipo',
 'tecnica',
 'family',
 'history',
 'symtomatic',
 'prosthesis',
 'birads',
 'density_mammo',
 'calcifications_benign',
 'density_echo',
 'ganglio_mamo',
 'lymph_benign',
 'lymph_suspicious',
 'parenchymal_distortion',
 'simple_cyst',
 'ductal_ectasia',
 'nodules_echo_num',
 'nodules_echo_description',
 'nodules_echo_shape',
 'nodules_echo_margin',
 'nodules_echo_echogenicity',
 'nodules_echo_location',
 'nodules_echo_size',
 'nodules_echo_known',
 'nodules_echo_stable']

In [7]:
model_name="biogpt"+"51e535e65"+"no_copy"+"description"+"16batch"+"final_answer_freeze"
for tipo in questions:
    print(tipo)
    with open(f"results_dic_{tipo}/{model_name.split("/")[-1]}.pkl", "wb") as file:
        pickle.dump(output_dic[tipo], file)
   

age
tipo
tecnica
family
history
symtomatic
prosthesis
birads
density_mammo
calcifications_benign
density_echo
ganglio_mamo
lymph_benign
lymph_suspicious
parenchymal_distortion
simple_cyst
ductal_ectasia
nodules_echo_num
nodules_echo_description
nodules_echo_shape
nodules_echo_margin
nodules_echo_echogenicity
nodules_echo_location
nodules_echo_size
nodules_echo_known
nodules_echo_stable


In [9]:
devices = cuda.gpus
dev=devices.current
cuda.select_device(dev.id)
cuda.close()

In [8]:
accuracies 4 epochs 1 e-5
[0.9952830188679245, 0.9764150943396226, 0.9764150943396226]

[0.9952830188679245, 0.9764150943396226, 0.9764150943396226]

In [12]:
acc,acc_std
solo un fallo

(0.9952830188679245, 0.013636363636363624)

In [ ]:
2 epoch eta bikoitza
(0.9811320754716981, 0.026424662271916612)

In [ ]:
con 8 epochs

In [ ]:
con 4 epochs pero añadiendo los datos sin etiqueta.

In [10]:
5 epochs 1e-5
" Important! We do not want biopsy, BAG, post BAG, nodal staging ultrasound or BI-RADS 6 (Cancer patients) reports. Answer 'no' if it is any of these. answer: "
(0.9811320754716981, 0.022268088570756142)

(0.9811320754716981, 0.022268088570756142)